In [1]:
# This script is for RF, socio variables
# Take Dataset3 LA for example

In [10]:
# Packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [11]:
X_socio_test_passed_1 = pd.read_csv("../Data/01 Processed Dataset/Socio_LA.csv") #input
X_socio_test_passed = X_socio_test_passed_1.set_index('GEOID')
y = X_socio_test_passed.pop('obesity_cr')

In [12]:
X_socio_test_passed.head()

,% Black,% Ame Indi and AK Native,% Asian,% Nati Hawa and Paci Island,% Hispanic or Latino,% male,% married,% age 18-29,% age 30-39,% age 40-49,...,% age >=60,% <highschool,median income,% unemployment,% below poverty line,% food stamp/SNAP,median value units built,median year units built,% renter-occupied housing units,population density
GEOID,,,,,,,,,,,,,,,,,,,,,
6037137504,0.009165,0.013025,0.015919,0.000000,0.030873,0.502653,0.550339,0.104197,0.093102,0.112880,...,0.372407,0.030710,158500,0.027916,0.043415,0.010086,1185500,1968,0.025937,540.339075
6037138000,0.071474,0.002354,0.071260,0.000000,0.084100,0.514659,0.604255,0.084314,0.126471,0.148727,...,0.296597,0.062192,128026,0.073945,0.082602,0.003363,832900,1966,0.172646,1045.182340
6037139200,0.026199,0.000000,0.143836,0.000000,0.227740,0.440925,0.406014,0.153767,0.171062,0.112158,...,0.215411,0.091443,67420,0.080120,0.103082,0.028545,560600,1971,0.429098,5070.897045
6037143200,0.091449,0.000000,0.061758,0.006888,0.128029,0.444893,0.334596,0.194299,0.259145,0.160095,...,0.128029,0.023082,74063,0.096330,0.108076,0.053769,693300,1977,0.712914,4400.511337
6037143300,0.070877,0.003863,0.072214,0.000000,0.188707,0.484695,0.436177,0.154532,0.158544,0.207875,...,0.203715,0.052264,87125,0.059385,0.057219,0.015098,810100,1968,0.595567,4068.456711


In [13]:
len(X_socio_test_passed)

947

In [14]:
y.head()

GEOID
6037137504    22.0
6037138000    23.4
6037139200    24.7
6037143200    23.5
6037143300    23.1
Name: obesity_cr, dtype: float64

In [15]:
def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

In [16]:
# Random forest 
y_rf_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

for train_index, test_index in ten_fold.split(X_socio_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_socio_test_passed.iloc[train_index], X_socio_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    rf = RandomForestRegressor(bootstrap=False, n_estimators=890, max_features=10, random_state=42) # input
    rf.fit(scaled_X_train, y_train)

    this_y_predict = rf.predict(scaled_X_test).flatten()

    y_rf_socio_predict = y_rf_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [ 23  30  39  44  59  63  67  70  72  76  86  88  96 107 120 136 139 165
 168 198 208 209 215 218 247 250 259 260 265 275 280 292 294 298 310 312
 327 331 332 333 363 365 377 388 394 439 449 453 457 464 465 478 481 495
 500 513 518 519 527 541 554 559 569 589 591 599 616 617 643 644 653 658
 673 679 684 695 708 753 762 767 777 778 783 792 804 816 826 850 858 874
 883 908 926 932 942]
TEST: [ 31  33  49  60  65  66  78 110 137 141 158 174 192 199 210 213 231 235
 254 261 266 286 296 302 306 307 309 311 314 316 326 328 334 342 352 361
 371 381 405 423 428 430 433 442 447 482 483 493 507 525 531 543 545 570
 572 576 594 602 605 606 615 625 630 685 689 694 707 732 735 736 737 746
 780 784 788 813 814 817 819 820 835 838 852 854 866 872 886 889 896 904
 910 925 941 944 945]
TEST: [  2   5   7  10  25  29  54  55  77  81  82  84  97 101 109 118 155 196
 204 211 227 228 239 244 281 318 319 321 323 344 346 350 355 357 380 398
 408 411 412 420 424 425 444 451 456 468 494 514 516 526 529 5

In [17]:
rf_socio_rmse = mean_squared_error(y_true , y_rf_socio_predict, squared=False) #False means return RMSE value
rf_socio_r2 = r2_score(y_true, y_rf_socio_predict)
avg_r_squared = avg_r_squared/10

In [18]:
# sociodemographic - estimators
print("RMSE of the RF model with sociodemographic predictors: "+str(rf_socio_rmse))
print("R2 of the RF model with sociodemographic predictors: "+str(rf_socio_r2)) # For R2, I took this one.
print("R2 of the RF model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the RF model with sociodemographic predictors: 1.2126231864089376
R2 of the RF model with sociodemographic predictors: 0.9502322232807352
R2 of the RF model with sociodemographic predictors (avg): 0.9496768654604237
